# Exploring the raw data

For this demo, we've generated some data and stored it in a Unity Catalog Volume. Volumes can store any type of file and can either be managed by Unity Catalog or connected to cloud storage. Lakeflow Declaritive Pipelines can automatically pick up new files and incrementally process data in a volume making your pipelines fast and efficient.

Let's start by taking a look at the contents of the `raw_data` volume.

**Note: this notebook is a simple Exploration Notebook, it's not part of our final Pipeline!**

Having a notebook on the side to test SQL queries interactively can be very handy to accelerate exploration and build your pipelines faster!

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-engineering&org_id=796524194907820&notebook=%2Fexplorations%2F01-Exploring-the-Data&demo_name=pipeline-bike&event=VIEW&path=%2F_dbdemos%2Fdata-engineering%2Fpipeline-bike%2Fexplorations%2F01-Exploring-the-Data&version=1">

In [0]:
import os

raw_data_volume = "/Volumes/main/dbdemos_pipeline_bike/raw_data/"

# Print out a list of directories in our raw_data volume and a few files from those directories
for table in os.listdir(raw_data_volume):
  print(table + "/")
  for file in os.listdir(raw_data_volume + table)[:3]:
    print("  " + file)
  print("  ...")


It looks like we've got a few directories here with `csv` and `json` files in them. Let's start by taking a look at the maintenance logs files using the SQL `read_files` function.

`read_files` supports several different file formats including `csv` and `json`. Take a look at the [Databricks documentation](https://docs.databricks.com/aws/en/sql/language-manual/functions/read_files) to see the available formats and options.

Additionally, using the `STREAM` keyword `read_files` can be used in streaming tables to ingest files into Delta Lake. `read_files` leverages Auto Loader when used in a streaming table query.



In [0]:
%sql
select * from read_files("/Volumes/main/dbdemos_pipeline_bike/raw_data/maintenance_logs/*.csv", format => "csv") limit 10

These files contains the field `issue_description` which is a free text field people can use to enter in a description of the issue they ran into while using a bike. Free text fields often include character sequences that may break CSV parsers. Let's do some data exploration on this data to see if we are processing it correctly.

Based on our knowledge of the system giving us this data, all the fields are required. Let's look at records where that's not the case.


Yup, it looks like there's some instances where the `issue_description` fields include a newline character. We can use `multiline => true` to tell `read_files` that records may span multiple lines and see if that fixes the issue. 

In [0]:
%sql
select * from read_files("/Volumes/main/dbdemos_pipeline_bike/raw_data/maintenance_logs/*.csv", format => "csv", multiline => true)
where maintenance_id is null or bike_id is null or reported_time is null or resolved_time is null


Let's do some quick spot checks on the `ride_logs` and `weather` files. The files in the `weather` directory are `json` files, so we need to make sure to use the `json` format option in `read_files`

In [0]:
%sql
select * from read_files("/Volumes/main/dbdemos_pipeline_bike/raw_data/rides/*.csv", format => "csv") limit 10

In [0]:
%sql
select * from read_files("/Volumes/main/dbdemos_pipeline_bike/raw_data/weather/*.json", format => "json") limit 10

In [0]:
%sql
select * from read_files("/Volumes/main/dbdemos_pipeline_bike/raw_data/customers_cdc/*.parquet", format => "parquet") limit 10


### Understanding Change Data Capture (CDC) with AUTO CDC

AUTO CDC is a declarative API in Lakeflow Declarative Pipelines that simplifies change data capture processing. 

Key benefits of AUTO CDC:
- **Automatic ordering**: Handles records that arrive out of chronological order
- **Built-in SCD support**: Easily implement Type 1 or Type 2 slowly changing dimensions
- **Declarative syntax**: Simple SQL-based configuration without complex merge logic
- **Operation handling**: Supports INSERT, UPDATE, DELETE, and TRUNCATE operations

Let's explore the distribution of CDC operations to understand the types of changes happening to customer data:

In [0]:
%sql
select 
  operation,
  count(*) as count,
  round(count(*) * 100.0 / sum(count(*)) over(), 1) as percentage
from read_files("/Volumes/main/dbdemos_pipeline_bike/raw_data/customers_cdc/*.parquet", format => "parquet") 
group by operation
order by count desc

### Next: Building our Declarative Pipeline
We now have a good idea of our raw data and the queries we'll have to do!

It's time to start building our pipeline!

If you want to know more about Streaming Tables and Materialized views, open the [00-pipeline-tutorial notebook]($../transformations/00-pipeline-tutorial).

If you know what you're doing, feel free to jump to the [01-bronze.sql]($../transformations/01-bronze.sql), [02-silver.sql]($../transformations/02-silver.sql) or [03-gold.sql]($../transformations/03-gold.sql) file!

### Alternative: learn how to track your Declarative Pipeline data quality

Lakeflow Declarative Pipelines makes it easy to track your data quality and set alerts when something is wrong! Open the [02-Pipeline-event-monitoring]($./02-Pipeline-event-monitoring) notebook for more details.